In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("..\examples"))

from __future__ import division,print_function

import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

Using Theano backend.


In [10]:
pwd = str(os.getcwd()).split("\\")
pwd.pop()
pwd.pop()
spwd = "/".join(pwd)
path = spwd + "/data/dogscats/"
#path = spwd + "/data/dogscats/sample/"

In [11]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=8

In [12]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

## 微调(finetune)模型

每次抓取一些图片用于训练(train)和验证(valid).  
对于vgg, 样本必须分类放好, 比如:  
data\train\cats\*.jpg  
data\train\dogs\*.jpg  

In [13]:
# 初始化模型并抓取图片
vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
# finetune用于更新vgg16的各种类别(self.classes), 即加入线性层, 使得输出形态与batches的形态一致
# fit则是训练这个线性层, 使得结果与数据一致
# 没有这两步, vgg则会根据原有的1000个类别, 在下一步中进行预测
vgg.finetune(batches)

In [ ]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
 5776/23000 [======>.......................] - ETA: 10830s - loss: 0.1930 - acc: 0.9595

## 生成结果

In [8]:
# 注意这里test目录下还要有一层目录, 再存放待识别的图片, 不然会找不到class
# 以及vgg.test返回两个值, 一个是get_batches的结果, 另一个是predict的结果:
# test_batches = self.get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
# return test_batches, self.model.predict_generator(test_batches, test_batches.nb_sample)

batches, preds = vgg.test(path+'test/', batch_size = batch_size*2)

Found 8 images belonging to 1 classes.


In [9]:
print(preds)

[[ 0.9628  0.0372]
 [ 0.9739  0.0261]
 [ 0.9697  0.0303]
 [ 0.9834  0.0166]
 [ 0.763   0.237 ]
 [ 0.1474  0.8526]
 [ 0.9982  0.0018]
 [ 0.9965  0.0035]]


In [25]:
preds.shape

(2L, 1000L)

In [5]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

## 储存和读取模型参数

In [15]:
vgg.model.save_weights(model_path+'finetune1.h5')

In [7]:
vgg = Vgg16()
vgg.model.load_weights(model_path+'finetune1.h5')